In [14]:
# here, we combine all .csv files into the single file total_state_data.csv

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df1 = pd.read_csv('../../SharedData/dataset-generation-final/population-1990-2020-final.csv')
df2 = pd.read_csv('../../SharedData/dataset-generation-final/monthly_emissions_1990_2024')
df3 = pd.read_csv('../../SharedData/dataset-generation-final/monthly-weather-1990-2019-final.csv')
df4 = pd.read_csv('../../SharedData/dataset-generation-final/gdp-1997-2023-final.csv')
df5 = pd.read_csv('../../SharedData/dataset-generation-final/energy_final.csv')
df6 = pd.read_csv('../../SharedData/dataset-generation-final/power_plant_count.csv')
df7 = pd.read_csv('../../SharedData/dataset-generation/state-areas.csv')
df8 = pd.read_csv('../../SharedData/dataset-generation-final/energy-by-source-final.csv')

In [15]:
# these lists are used to fix abbreviations in state-areas.csv

state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
    "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
    "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
    "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
name_dict = {state_names[i]:abbreviations[i] for i in range(len(state_names))}

In [16]:
# remove duplicate indexing columns

df1 = df1.drop('Unnamed: 0', axis=1)
df4 = df4.drop('Unnamed: 0', axis=1)
df6 = df6.drop('Unnamed: 0', axis=1)

# remove duplicate year/month columns

df6 = df6.drop(['year','month'], axis=1)
df8 = df8.drop(['year','month'], axis=1)

# fix datetime formatting

df2['date'] = df2['date'].apply(pd.to_datetime)
df2['date'] = df2['date'].dt.strftime('%Y-%m')

# fix abbreviations in state-areas.csv and make it into a dict

df7['state'] = df7['state'].map(name_dict)
df7_dict = df7.set_index('state')['area'].to_dict()

In [17]:
# merge all the dfs on 'state' and 'date'

total_df = df1.merge(df2, on=['state','date'], how='right').merge(df3, on=['state','date'], how='left').merge(df4, on=['state','date'], how='left').merge(df5, on=['state','date'], how='left').merge(df6, on=['state','date'], how='left').merge(df8,on=['state','date'], how='left') 

In [18]:
# remove DC (not a state), HI, AK (states with missing CO2 data)

total_df = total_df[~total_df.state.isin(['DC','HI','AK'])]

In [19]:
# fix datetime formatting

total_df['date']=pd.to_datetime(total_df['date'])

In [20]:
# add column for sq ft of each state to calculation population density

total_df['state_sq_ft']= total_df['state'].map(df7_dict)

# add columns for monthly/yearly population density

total_df['monthly_pop_density']= total_df['state_sq_ft']/total_df['monthly_population']
total_df['yearly_pop_density']= total_df['state_sq_ft']/total_df['yearly_population']

In [21]:
# create column for simple moving avg

total_df['monthly_emissions_sma']= np.nan

for state in abbreviations:
    total_df.loc[total_df.state==state,'monthly_emissions_sma'] = total_df.loc[total_df.state==state,'monthly_emissions'].rolling(window=12).mean()

In [22]:
# reorder the columns for readability

total_df = total_df[['state', 'state_sq_ft', 
        'year', 'month', 'date', 
        'monthly_population', 'yearly_population', 'monthly_pop_density', 'yearly_pop_density',
        'monthly_emissions', 'monthly_emissions_sma',
        'prcp', 'snow', 'tavg', 
        'gdp_rel_2017', 'gdp_rel_2017_interp', 'yearly_gdp_rel_2017',  
        'monthly_energy_prod', 'yearly_energy_prod',
        'monthly_energy_use', 'yearly_energy_use',
        'monthly_energy_flow', 'yearly_energy_flow',         
        'monthly_num_plants', 'yearly_num_plants', 
        'monthly_energy_total', 'yearly_energy_total',
        'monthly_energy_renew', 'yearly_energy_renew', 'monthly_renew_pct', 'yearly_renew_pct',       
        'monthly_energy_fossil', 'yearly_energy_fossil', 'monthly_fossil_pct', 'yearly_fossil_pct' 
        ]]

In [23]:
total_df.to_csv('../../SharedData/total_state_data.csv',index=False)